# Clasificador de imágenes

In [ ]:
import pandas as pd
import os
import numpy as np
import zipfile
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 1. Cargamos el conjunto de datos

In [ ]:
# Ruta al archivo ZIP
zip_path = "../dogs-vs-cats.zip" 
extract_to = "../data/raw"       

# Extraer el archivo ZIP
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Archivo extraído correctamente en: {extract_to}")
else:
    print(f"El archivo ZIP no existe en la ruta: {zip_path}")

In [ ]:
# Ruta al archivo ZIP
zip_path = "../data/raw/train.zip" 
extract_to = "../data/raw"       

# Extraer el archivo ZIP
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Archivo extraído correctamente en: {extract_to}")
else:
    print(f"El archivo ZIP no existe en la ruta: {zip_path}")

In [ ]:
# Ruta al archivo ZIP
zip_path = "../data/raw/test1.zip" 
extract_to = "../data/raw"       

# Extraer el archivo ZIP
if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    print(f"Archivo extraído correctamente en: {extract_to}")
else:
    print(f"El archivo ZIP no existe en la ruta: {zip_path}")

# 2. Visualización y preprocesamiento de imágenes

In [ ]:
# Carpetas de entrada y salida
input_dir = "../data/raw/train"
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

# Procesar imágenes
image_size = (255, 255)
categories = ['cat', 'dog']

# Crear carpetas de salida para cada categoría
for category in categories:
    os.makedirs(os.path.join(output_dir, category), exist_ok=True)

# Redimensionar y mover las imágenes
for filename in os.listdir(input_dir):
    category = 'cat' if 'cat' in filename else 'dog'
    img_path = os.path.join(input_dir, filename)
    img = cv2.imread(img_path)
    if img is not None:
        img_resized = cv2.resize(img, image_size)
        save_path = os.path.join(output_dir, category, filename)
        cv2.imwrite(save_path, img_resized)

print("Todas las imágenes han sido redimensionadas y clasificadas.")

In [ ]:
cat_images = [os.path.join(output_dir, "cat", img) for img in os.listdir(os.path.join(output_dir, "cat"))[:9]]
dog_images = [os.path.join(output_dir, "dog", img) for img in os.listdir(os.path.join(output_dir, "dog"))[:9]]

def plot_images(image_paths, title):
    plt.figure(figsize=(10, 10))
    for i, img_path in enumerate(image_paths):
        plt.subplot(3, 3, i + 1)
        img = cv2.imread(img_path)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
    plt.suptitle(title)
    plt.show()

plot_images(cat_images, "Ejemplos de Gatos")
plot_images(dog_images, "Ejemplos de Perros")

# 3. Construir la Red Neuronal Convolucional (CNN)

In [ ]:
# Crear el modelo
model = Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(200, 200, 3)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPool2D((2, 2)),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPool2D((2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(2, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
train_dir = "../data/processed"
datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_directory(
    train_dir, target_size=(200, 200), batch_size=32, class_mode='categorical', subset='training')

validation_generator = datagen.flow_from_directory(
    train_dir, target_size=(200, 200), batch_size=32, class_mode='categorical', subset='validation')

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=1,
    validation_data=validation_generator
)

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Precisión en el conjunto de validación: {accuracy}%")

In [ ]:
checkpoint = ModelCheckpoint("../models/img-classificator", monitor = "val_accuracy", verbose = 1, save_best_only = True, save_weights_only = False, mode = "auto")
early = EarlyStopping(monitor = "val_accuracy", patience = 3, verbose = 1, mode = "auto")
hist = model.fit(train_data, steps_per_epoch = 100, validation_data = test_data, validation_steps = 10, epochs = 3, callbacks = [checkpoint, early])

# 4. Visualización de los datos

In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history["val_accuracy"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])

plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "Loss", "Validation Loss"])

plt.show()

In [ ]:
img = image.load_img("../data/raw/test/9.jpg", target_size = (200, 200))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis = 0)
saved_model = load_model("../models/vgg16_1.h5")
output = saved_model.predict(img)
if output[0][0] > output[0][1]:
    print("cat")
else:
    print("dog")